# Вопросы
1. Теория
- нужно ли разбивать на предложения? Сейчас в примере может быть и по 5 предложений. Но если разбивать у нас уйдут случаи разрыва: "Я ухожу. Потому что я устала"
2. Комп
- active learning
- смотреть на время глагола (док с примерами конструкций)

Комментарий преподавателя по программированию (я взяла итоговым проектом эту тему): 

- для вашей специфической задачи стоит лучше продумать датасет. Что нужно сделать, чтобы было лучше: получать разметку еще каким-то парсером (из презентации не поняли, чем плохи зависимостные парсеры), попробовать отсмотреть предложения, где один парсер находит инсубординацию, а другой -нет. Если там действительно инсубординация, то это интересные сложные для одного из парсеров примеры и ценно на них научиться правильно работать, если там нет инсубординации - то это ваши hard negative  примеры (то есть не банальные отрицательные). Пока же, насколько было понятно из презентации, классификатор должен просто научиться отличать предложение из одной клаузы от сложного предложения. 
+ посоветовали реализовать active learning для обучения на семантические (прагматические?) типы



Ссылки:
- [Что-то типо маленького корпуса, где все предложения точно инсубординация,тк были взяты из статей](https://drive.google.com/file/d/1FUKg9CM6WGxmkkzdcy7yt6wnDfAQpGIF/view?usp=sharing)
- [Тут попытка найти инсубординацию по шаблону, опираясь на структцру составляющих](https://colab.research.google.com/drive/1Td-yuykoYDqVoN6j4IGyewu4HzKz5389?usp=sharing)
- [Классификация  Knn, svm+naive Bayes](https://colab.research.google.com/drive/1M59XMzwrmJWcpcCSFHKkmohNXw45AB5X#scrollTo=81LBNEe03vJB)
- [Классификация  Предсказание BILSTM CHAR и Log Reg на Bert Features](https://colab.research.google.com/drive/1yDxrJRe91Nwp3HydUp9m7UDv0u6kVwT-?usp=sharing)
- [Eще тетадка](https://colab.research.google.com/drive/1Td-yuykoYDqVoN6j4IGyewu4HzKz5389?usp=sharing)

Далее кратко описывается, что было сделано:

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ВКР/proga/DATA/data_full.csv').drop(columns=['regions'])
data.head()

,link_id,subreddit_id,uid,comment_id,parent_id,score,create_utc,text,dependence_structure,tree_structure
0,8r1kz,2qhjz,1688932,c0a62uj,8r1kz,3,1244576002,Ironie : l'article disant qu'on est plus capab...,"[[{'id': 1, 'text': 'Ironie', 'lemma': 'ironie...","['(SENT (NP (NC Ironie)) (PONCT :))', ""(SENT (..."
1,8r1kz,2qhjz,786883,c0a6lmb,c0a62uj,1,1244621120,"Moi-même, j'ai dû me forcer pour arriver jusqu...","[[{'id': 1, 'text': 'Moi-même', 'lemma': 'loi-...","[""(SENT (NP (PRO+ (PRO Moi) (PONCT -) (ADJ mêm..."
2,8sncs,2qhjz,390497,c0aawpk,8sncs,1,1245076061,Service qui sera rendu au contribuable pour la...,"[[{'id': 1, 'text': 'Service', 'lemma': 'servi...",['(SENT (NP (NC Service) (Srel (NP (PROREL qui...
3,8sncs,2qhjz,32884,c0aaxba,c0aawpk,3,1245077396,Eeeeh oui ! 70 millions pour une loi qui aura ...,"[[{'id': 1, 'text': 'Eeeeh', 'lemma': 'Eeeeh',...","['(SENT (I Eeeeh) (I oui) (PONCT !))', '(SENT ..."
4,8v13c,2qhjz,796919,c0aj3ov,8v13c,2,1245830384,Est-ce qu'elle a vraiment commis des crimes qu...,"[[{'id': 1, 'text': 'Est', 'lemma': 'être', 'u...","[""(SENT (VN (V Est) (CLS -ce)) (Ssub (CS qu') ..."


In [ ]:
# Берем первую тысячу примеров + уже добавили структуру зависимости (колонка dependence_structure) и структуру составляющих (колонка tree_structure)
len(data)

1000

In [ ]:
# Посмотрим на первый пример в датафрейма
data.text[0]

"Ironie : l'article disant qu'on est plus capable de se concentrer plus de 2 pages, en fait 6 !\nJ'ai pas pu aller plus loin que la page 3, même si c'était intéressant !"

## Смотрим на струтуру составляющих и выбираем нужные конструкции

In [ ]:
!pip install stanza
import stanza
stanza.download('fr') # download French model
nlp_stanza = stanza.Pipeline('fr') # initialize French neural pipeline

In [ ]:
!pip install --upgrade spicy
!python -m spacy download fr_core_news_md
import fr_core_news_md
nlp = fr_core_news_md.load()

In [ ]:
!pip install benepar
from nltk import Tree
import benepar
benepar.download('benepar_fr2')
import warnings
warnings.filterwarnings("ignore")
import spacy
from benepar.spacy_plugin import BeneparComponent
nlp.add_pipe(BeneparComponent('benepar_fr2'))

In [ ]:
# находим предложеня с подчинительным сюзом - как видим, их половина
idxs_sconj = []
for i, text in enumerate(data.dependence_structure):
  for sent in eval(text):
    for word in sent:
      try:
        if word['upos'] == 'SCONJ':
          if i not in idxs_sconj:
            idxs_sconj.append(i)
      except:
        continue

len(idxs_sconj)

558

In [ ]:
si = []
que = []
comme = []
lorsque = []
quand = []
sinon = []
puisque = []

for i in idxs_sconj:
  for sent in eval(data.dependence_structure[i]):
    for word in sent:
      try:
        if word['upos'] == 'SCONJ':
          if word['lemma'] == 'si':
            si.append(i)
          if word['lemma'] == 'que':
            que.append(i)
          if word['lemma'] == 'comme':
            comme.append(i)
          if word['lemma'] == 'lorsque':
            lorsque.append(i)
          if word['lemma'] == 'quand':
            quand.append(i)
          if word['lemma'] == 'sinon':
            sinon.append(i)
          if word['lemma'] == 'puisque':
            puisque.append(i)
      except:
        continue

print('si-----',len(si))
print('que-----',len(que))
print('comme-----',len(comme))
print('lorsque-----',len(lorsque))
print('quand-----',len(quand))
print('sinon-----',len(sinon))
print('puisque-----',len(puisque))

si----- 162
que----- 796
comme----- 54
lorsque----- 13
quand----- 77
sinon----- 5
puisque----- 13


In [ ]:
# посмотрим на примеры с подчинительным союзом
for i in idxs_sconj:
  print(i,'-----',data.text[i])
  print('\n')

0 ----- Ironie : l'article disant qu'on est plus capable de se concentrer plus de 2 pages, en fait 6 !
J'ai pas pu aller plus loin que la page 3, même si c'était intéressant !


4 ----- Est-ce qu'elle a vraiment commis des crimes qui méritent **soixante** ans de prison? J'en doute...


7 ----- Si ils ne sortent pas des idées pertinentes d'ici un an, ils sont morts. La seul chose que produit le PS depuis plusieurs années maintenant, c'est:

1. Des feuilletons pathétiques de divisions internes.
2. Un show people (ségolène royal).
3. De l'antisarkozysme primaire.
4. De l'obstruction parlementaire.


9 ----- Donner le PS à une people sans idées? Quelle drôle d'idée! Non, il faut arrêter les grandes déclarations et commencer à contribuer par des idées au débat national. Evidemment, si les médias relayaient ce genre d'initiatives plutôt que les frasques de ségolène, ça aiderait...


10 ----- à quand la taxe Tobin?



11 ----- Le vrai problème ce sont les bulles spéculatives et la taxe Tobin 

In [3]:
# показывает дерево предложения
def make_tree(text):
  doc = nlp(text)
  sents = list(doc.sents)
  for sent in sents:
    print(sent._.parse_string)
    result = Tree.fromstring(str(sent._.parse_string)).pretty_print()
  pass

# строит структуру составляющих предложения
def make_structure(text):
    doc = nlp(text)
    sents = list(doc.sents)
    result = [sent._.parse_string for sent in sents]
    return result

# парсим структуру составляющих + берем только уровень 1
def parenthetic(string):
    result = []
    stack = []
    for i, c in enumerate(string):
        if c == '(':
            stack.append(i)
        elif c == ')' and stack:
            start = stack.pop()
            if len(stack) == 1:
              result.append(string[start + 1: i].split(' ')[0])
    return result 

def dep_struct(text):
  doc = nlp_stanza(text)
  list_doc = doc.to_dict()
  return list_doc

In [ ]:
# В датафрейм я уже добавила структуру зависимости, но мне кажется, это не помогает в классификации
# Посмотрим на структуру зависимости предложения с инсубординацией и без (примеры выдуманые, не из датасета)
sub = 'Si tu savais ce que moi j’étais contente, tu serais étonné!'
insub = '''Si tu savais ce que moi j'étais content!'''

In [ ]:
dep_struct(sub)

[[{'deprel': 'mark',
   'head': 3,
   'id': 1,
   'lemma': 'si',
   'misc': 'start_char=0|end_char=2',
   'ner': 'S-MISC',
   'text': 'Si',
   'upos': 'SCONJ'},
  {'deprel': 'nsubj',
   'feats': 'Number=Sing|Person=2|PronType=Prs',
   'head': 3,
   'id': 2,
   'lemma': 'il',
   'misc': 'start_char=3|end_char=5',
   'ner': 'O',
   'text': 'tu',
   'upos': 'PRON'},
  {'deprel': 'advcl',
   'feats': 'Mood=Ind|Number=Sing|Person=1|Tense=Imp|VerbForm=Fin',
   'head': 13,
   'id': 3,
   'lemma': 'savoir',
   'misc': 'start_char=6|end_char=12',
   'ner': 'O',
   'text': 'savais',
   'upos': 'VERB'},
  {'deprel': 'obj',
   'feats': 'Gender=Masc|Person=3|PronType=Dem',
   'head': 3,
   'id': 4,
   'lemma': 'ce',
   'misc': 'start_char=13|end_char=15',
   'ner': 'O',
   'text': 'ce',
   'upos': 'PRON'},
  {'deprel': 'mark',
   'feats': 'PronType=Rel',
   'head': 9,
   'id': 5,
   'lemma': 'que',
   'misc': 'start_char=16|end_char=19',
   'ner': 'O',
   'text': 'que',
   'upos': 'PRON'},
  {'depr

In [ ]:
dep_struct(insub)

[[{'deprel': 'nsubj',
   'feats': 'Gender=Fem|Number=Sing|Person=3|PronType=Prs',
   'head': 6,
   'id': 1,
   'lemma': 'il',
   'misc': 'start_char=0|end_char=4',
   'ner': 'O',
   'text': 'Elle',
   'upos': 'PRON'},
  {'deprel': 'advmod',
   'feats': 'Polarity=Neg',
   'head': 6,
   'id': 2,
   'lemma': 'ne',
   'misc': 'start_char=5|end_char=7',
   'ner': 'O',
   'text': "n'",
   'upos': 'ADV'},
  {'deprel': 'cop',
   'feats': 'Mood=Ind|Number=Sing|Person=3|Tense=Imp|VerbForm=Fin',
   'head': 6,
   'id': 3,
   'lemma': 'être',
   'misc': 'start_char=7|end_char=12',
   'ner': 'O',
   'text': 'était',
   'upos': 'AUX'},
  {'deprel': 'advmod',
   'feats': 'Polarity=Neg',
   'head': 6,
   'id': 4,
   'lemma': 'pas',
   'misc': 'start_char=13|end_char=16',
   'ner': 'O',
   'text': 'pas',
   'upos': 'ADV'},
  {'deprel': 'advmod',
   'head': 6,
   'id': 5,
   'lemma': 'très',
   'misc': 'start_char=17|end_char=21',
   'ner': 'O',
   'text': 'très',
   'upos': 'ADV'},
  {'deprel': 'root',


In [ ]:
# теперь посмотрим на структуру составляющих  и дерево
print(make_tree(sub))
print('\n','--------','\n')
print(make_tree(insub))

(SENT (Ssub (Ssub (CS Si) (Sint (VN (CLS tu) (V savais)) (NP (PRO ce) (Ssub (NP (CS que)) (NP (PRO moi)) (VN (CLS j’) (V étais)) (AP (ADJ contente)))))) (PONCT ,)) (VN (CLS tu) (V serais) (VPP étonné)) (PONCT !))
                                SENT                                                 
                         ________|________________________________________________    
                       Ssub                                         |             |  
               _________|_________________________________          |             |   
             Ssub                                         |         |             |  
  ____________|_________                                  |         |             |   
 |                     Sint                               |         |             |  
 |        ______________|____                             |         |             |   
 |       |                   NP                           |         |             |  
 |       

In [5]:
make_tree('Si tu savai ce que moi j’étais contente!')

(SENT (Ssub (CS Si) (Sint (VN (CLS tu) (V savais)) (NP (PRO ce) (Srel (NP (PROREL que)) (V moi))))) (VN (CLS j’) (V étais)) (AP (ADJ contente)) (PONCT !))
                         SENT                                       
               ___________|______________________________________    
             Ssub                            |           |       |  
  ____________|_____                         |           |       |   
 |                 Sint                      |           |       |  
 |        __________|_____                   |           |       |   
 |       |                NP                 |           |       |  
 |       |           _____|_____             |           |       |   
 |       |          |          Srel          |           |       |  
 |       |          |      _____|____        |           |       |   
 |       VN         |     NP         |       VN          AP      |  
 |    ___|____      |     |          |    ___|____       |       |   
 CS CLS    

In [ ]:
# единственные очевидные случаи инсубординации выглядят вот так
# когда предложение состоит только  из "подчиненной" клаузы Ssub   
perfect_insub = '''Si seulement il avait pu essayer!''' # Если бы только он попробовал!
print(make_tree(perfect_insub))

(SENT (Ssub (CS Si) (Sint (ADV seulement) (VN (CLS il) (V avait) (VPP pu)) (VPinf (VN (VINF essayer))))) (PONCT !))
                    SENT                  
                _____|_________________    
              Ssub                     |  
  _____________|_____                  |   
 |                  Sint               |  
 |       ____________|__________       |   
 |      |            |        VPinf    |  
 |      |            |          |      |   
 |      |            VN         VN     |  
 |      |       _____|____      |      |   
 CS    ADV    CLS    V   VPP   VINF  PONCT
 |      |      |     |    |     |      |   
 Si seulement  il  avait  pu essayer   !  

None


In [ ]:
# парсим структуру составляющих
def parenthetic_full(string):
    result = []
    stack = []
    for i, c in enumerate(string):
        if c == '(':
            stack.append(i)
        elif c == ')' and stack:
            start = stack.pop()
            result.append((len(stack),string[start + 1: i].split(' ')[0]))
    return result
 
sorted(parenthetic_full(make_structure(perfect_insub)[0]))

[(0, 'SENT'),
 (1, 'PONCT'),
 (1, 'Ssub'),
 (2, 'CS'),
 (2, 'Sint'),
 (3, 'ADV'),
 (3, 'VN'),
 (3, 'VPinf'),
 (4, 'CLS'),
 (4, 'V'),
 (4, 'VN'),
 (4, 'VPP'),
 (5, 'VINF')]

In [ ]:
parenthetic(make_structure(perfect_insub)[0]) # Берем только 1 уровень

['Ssub', 'PONCT']

In [ ]:
from tqdm.auto import tqdm

In [ ]:
insub_list = []
for i in tqdm(idxs_sconj):
  for sent in eval(data.tree_structure[i]):
    if parenthetic(sent) == ['Ssub', 'PONCT']:
      insub_list.append(i)

In [ ]:
for i in insub_list:
  print(data.text[i])
  print('\n','--------','\n')

Surtout quand la manière de faire obtenir le bac à tout le monde est de baisser le niveau du bac...

 -------- 

Bah si quand même, ça demande minimum d'effort de se connecter à un réseau WEP, alors que sans protection c'est automatique. En obligeant les gens à protéger leur réseau, même si la protection est mauvaise, on élimine l'immense majorité des squattages de ouiphi.

 -------- 

>Ce n'est pas du gaspillage que d'acheter un smartphone.

Ben si, c'est du gaspillage complet, puisqu'ils en ont tous déjà un! Les grosses voitures de fonction on pourrait sans doute aussi s'en passer, mais pour le reste, je ne suis pas d'accord. Tu as déjà essayé de réguler l'agriculture européenne? Non? Je suis d'accord, tout n'est pas parfait, mais personnellement, je suis plus que content qu'on ait réussi à garder une agriculture forte en Europe. Sans la PAC, on serait probablement complètement dépendants de quelques pays étrangers pour notre approvisionnement.

Quand aux traducteurs, quand on voit l

In [ ]:
parenthetic(make_structure(data.text[0]))

[]

In [ ]:
make_structure(data.text[0])

['(SENT (NP (NC Ironie)) (PONCT :))',
 "(SENT (NP (DET l') (NC article) (VPpart (VN (VPR disant)) (Ssub (CS qu') (Sint (VN (CLS on) (V est)) (AP (ADV plus) (ADJ capable) (PP (P de) (VPinf (VN (CLR se) (VINF concentrer)) (NP (P+ (ADV plus) (P de)) (NP (DET 2) (NC pages)))))))))) (PONCT ,) (ADV+ (P en) (NC fait)) (NP (PRO 6)) (PONCT !) (PONCT \n))",
 "(SENT (VN (CLS J') (V ai) (ADV pas) (VPP pu)) (VPinf (VN (VINF aller)) (AdP (ADV plus) (ADV loin) (Ssub (CS que) (NP (DET la) (NC page) (ADJ 3))))) (PONCT ,) (Ssub (CS+ (ADV même) (CS si)) (Sint (VN (CLS c') (V était)) (AP (ADJ intéressant)))) (PONCT !))"]

In [ ]:
# попробуем достать только предложения состоящие из Ssub
